In [1]:
import os
import sys
import git
import pathlib

import random
import time
import glob

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist


PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs import utils
from libs.ga.ga_utils import get_chromosomes

In [2]:
# set seed
this_seed = 84376
tf.random.set_seed(this_seed)
np.random.seed(this_seed)
random.seed(this_seed)
os.environ['PYTHONHASHSEED']=str(this_seed)

In [3]:
# Tensorflow Limit GPU usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        

In [4]:
# Extract model
# parser = argparse.ArgumentParser()
# parser.add_argument('model_instance')
# parser.add_argument('error_profile_tag')
# args= parser.parse_args()
# model_instance = args.model_instance
# error_profile_tag = args.error_profile_tag

In [5]:
experiment = "mnist32-cnn_1024_256_64-7278--LIM_05-2188"
print(experiment)
model_instance, error_profile_tag = experiment.split("--")

mnist32-cnn_1024_256_64-7278--LIM_05-2188


In [6]:
error_param = -1 # bitflips
optim_type = "all"
META_RS = "vanilla"
print("ERR_PARAM:\t", error_param)
print("OPTIM_TYPE:\t", optim_type)
print("META_RS:\t", META_RS)

ERR_PARAM:	 -1
OPTIM_TYPE:	 all
META_RS:	 vanilla


In [7]:
# Prepare dataset
# Combine test and train images together into one dataset
DATASET_PATH = str(pathlib.Path(PROJ_ROOT_PATH / "datasets" / "mnist.npz" ))
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path=DATASET_PATH)
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0  

all_images =np.concatenate([train_images, test_images], axis=0)
all_labels =np.concatenate([train_labels, test_labels], axis=0)
all_images = np.expand_dims(all_images, axis=-1)

# resize the input shape , i.e. old shape: 28, new shape: 32
image_x_size = 32
image_y_size = 32
all_images = tf.image.resize(all_images, [image_x_size, image_y_size]) 

In [8]:
# Get model
dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)
model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_meta_type / model_type)
model_filename = model_instance + ".h5"
model_file = pathlib.Path(model_folder/ model_filename)

In [9]:
# Load model
model = tf.keras.models.load_model(model_file)

In [10]:
# Load error profile
error_profile_folder = pathlib.Path(PROJ_ROOT_PATH / "error_profiles")
error_profile_filename = error_profile_tag + ".npy"
error_profile_file = pathlib.Path(error_profile_folder/ error_profile_filename)
error_profile = np.load(error_profile_file)

In [11]:
# Batchsize for evaluation
EVAL_BATCHSIZE = 128

In [12]:
# Load test set
# Testing with only the last 12_80 images
test_set = (all_images[-12_80:], all_labels[-12_80:])
# test_set = (all_images, all_labels)

In [13]:
def extract_best_worst_chromosomes(optim_type, # (q1, q2, q3, q4)
                                   META_RS, # (vanilla, qent, qmean)
                                   layer, 
                                   error_param, 
                                   model_instance, 
                                   error_profile_tag, 
                                   this_seed):
    
    META_RS = "vanilla" # type of dataset segregation (qent, qmean)
    archerr_type = "lenet_3hidden_ERRexpbitflips"
    meta_experiment_name = model_instance + '--' + error_profile_tag
    RS_experiment_type = optim_type + "-" + "RowShuffle_" + layer + "_" + archerr_type + "_" + str(error_param)
    ga_experiment_name = meta_experiment_name + '--ga_' + str(this_seed)
   
    # Folder to load log files
    logging_folder = pathlib.Path(PROJ_ROOT_PATH / "logging" / META_RS / archerr_type /  meta_experiment_name / RS_experiment_type )

    ga_logging_filename_tag = pathlib.Path(logging_folder/ ga_experiment_name)

    # ga outputs
    ga_output_files = glob.glob(str(ga_logging_filename_tag) + "*.log")

    # get chromosomes strings
    best_chromosome, worst_chromosome = get_chromosomes(ga_output_files)
    
    return(best_chromosome, worst_chromosome)

In [14]:
# get best and worst chromosomes for c0layer
layer = "c0layer"
c0layer_best_chromosome, c0layer_worst_chromosome = extract_best_worst_chromosomes(optim_type, 
                                                                                   META_RS,
                                                                                   layer, 
                                                                                   error_param, 
                                                                                   model_instance, 
                                                                                   error_profile_tag, 
                                                                                   this_seed)

# get best and worst chromosomes for h2layer
layer = "h2layer"
h2layer_best_chromosome, h2layer_worst_chromosome = extract_best_worst_chromosomes(optim_type, 
                                                                                   META_RS,
                                                                                   layer, 
                                                                                   error_param, 
                                                                                   model_instance, 
                                                                                   error_profile_tag, 
                                                                                   this_seed)

In [15]:
from libs.fitnessfns import eval_lenet_3hidden_ERRexpbitflips

In [16]:
# accuracy after error-injection to layers [c0, h2]
def eval_error_injected_with_shuffle(model, 
                                      layer_error_profiles, 
                                      ERR_PARAM,
                                      shuffle_orders,
                                      test_set,
                                      batchsize):
    # intialize to None
    error_profile_c0=None
    error_profile_h0=None
    error_profile_h1=None
    error_profile_h2=None
    error_profile_op=None

    [error_profile_c0,
    error_profile_h0,
    error_profile_h1,
    error_profile_h2,
    error_profile_op] = layer_error_profiles

    
    # intialize to None
    clayer0_shuffle_order=None
    hlayer0_shuffle_order=None
    hlayer1_shuffle_order=None
    hlayer2_shuffle_order=None
    oplayer_shuffle_order=None
                    
    [clayer0_shuffle_order,
    hlayer0_shuffle_order,
    hlayer1_shuffle_order,
    hlayer2_shuffle_order,
    oplayer_shuffle_order] = shuffle_orders
        
    acc_list = []
    for _ in range(3):
        acc, std = eval_lenet_3hidden_ERRexpbitflips(model,
                                                    error_profile_c0,
                                                    error_profile_h0,
                                                    error_profile_h1,
                                                    error_profile_h2,
                                                    error_profile_op,
                                                    ERR_PARAM,
                                                    clayer0_shuffle_order,
                                                    hlayer0_shuffle_order,
                                                    hlayer1_shuffle_order,
                                                    hlayer2_shuffle_order,
                                                    oplayer_shuffle_order,
                                                    test_set)
        acc_list.append(acc)
    return np.mean(acc_list)

In [17]:
error_profiles = [None, None, None, None, None]
ERR_PARAM = None
shuffle_orders = [None, None, None,None, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("Original ACC: ", eval_acc)

Original ACC:  0.98984375


In [18]:
# injecting in c0 layer only
error_profiles = [error_profile, None, None, None, None]
ERR_PARAM = -1
shuffle_orders = [None, None, None,None, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("Faulty ACC [Only c0]: ", eval_acc)

Faulty ACC [Only c0]:  0.5872395833333334


In [19]:
# injecting in c0 layer only with best GA optimization
error_profiles = [error_profile, None, None, None, None]
ERR_PARAM = -1
shuffle_orders = [c0layer_best_chromosome, None, None,None, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("GA ACC [Only c0 - best]: ", eval_acc)

GA ACC [Only c0 - best]:  0.8986979166666665


In [20]:
# injecting in c0 layer only with worst GA optimization
error_profiles = [error_profile, None, None, None, None]
ERR_PARAM = -1
shuffle_orders = [c0layer_worst_chromosome, None, None,None, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("GA ACC [Only c0 - worst]: ", eval_acc)

GA ACC [Only c0 - worst]:  0.8421875


In [21]:
# injecting in fc_2 layer only
error_profiles = [None, None, None, error_profile, None]
ERR_PARAM = -1
shuffle_orders = [None, None, None,None, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("Faulty ACC [Only h2]: ", eval_acc)

Faulty ACC [Only h2]:  0.9497395833333334


In [22]:
# injecting in fc_2 layer only with best GA optimization
error_profiles = [None, None, None, error_profile, None]
ERR_PARAM = -1
shuffle_orders = [None, None, None,h2layer_best_chromosome, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("GA ACC [Only h2 - best]: ", eval_acc)

GA ACC [Only h2 - best]:  0.9565972222222223


In [23]:
# injecting in fc_2 layer only with worst GA optimization
error_profiles = [None, None, None, error_profile, None]
ERR_PARAM = -1
shuffle_orders = [None, None, None,h2layer_worst_chromosome, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("GA ACC [Only h2 - worst]: ", eval_acc)

GA ACC [Only h2 - worst]:  0.9561631944444445


In [24]:
# injecting in c0 AND fc_2 layer
error_profiles = [error_profile, None, None, error_profile, None]
ERR_PARAM = -1
shuffle_orders = [None, None, None,None, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("Faulty ACC [c0 AND h2]: ", eval_acc)

Faulty ACC [c0 AND h2]:  0.5638020833333331


In [25]:
# injecting in c0 AND fc_2 layer with best GA optimization
error_profiles = [error_profile, None, None, error_profile, None]
ERR_PARAM = -1
shuffle_orders = [c0layer_best_chromosome, None, None,h2layer_best_chromosome, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("GA ACC [c0 AND h2] -best : ", eval_acc)

GA ACC [c0 AND h2] -best :  0.8689236111111112


In [26]:
# injecting in c0 AND fc_2 layer with worst GA optimization
error_profiles = [error_profile, None, None, error_profile, None]
ERR_PARAM = -1
shuffle_orders = [c0layer_worst_chromosome, None, None,h2layer_worst_chromosome, None]
test_set = test_set
batchsize = EVAL_BATCHSIZE
eval_acc  = eval_error_injected_with_shuffle(model, 
                                  error_profiles, 
                                  ERR_PARAM,
                                  shuffle_orders,
                                  test_set,
                                  batchsize)
print("GA ACC [c0 AND h2] -worst : ", eval_acc)

GA ACC [c0 AND h2] -worst :  0.8113715277777779
